[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/locality_sensitive_hashing_traditional/sparse_implementation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/locality_sensitive_hashing_traditional/sparse_implementation.ipynb)

# Sparse Implementation of LSH

Locality Sensitive Hashing (LSH) can be implemented for both sparse and dense vectors. In this notebook we will implement the algorithm for searching sparse vectors. We will be using *k-shingling* and *minhashing* to create our sparse vectors before applying them for search with LSH.

We start by defining a few sentences.

In [1]:
a = "flying fish flew by the space station"
b = "he will not allow you to bring your sticks of dynamite and pet armadillo along"
c = "he figured a few sticks of dynamite were easier than a fishing pole to catch an armadillo"

The first thing we do is create our shingles, we will use k-shingles where `k == 2`. For longer text it is recommended to create shingles where it is unlikely to produce matching shingles between non-matching text, `k` values of `7` to `11` would likely produce this outcome.

In [2]:
k = 2
for i in range(len(a) - k+1):
    print(a[i: i+k], end='|')

fl|ly|yi|in|ng|g | f|fi|is|sh|h | f|fl|le|ew|w | b|by|y | t|th|he|e | s|sp|pa|ac|ce|e | s|st|ta|at|ti|io|on|

These are our shingles, however, we must remove duplicate values as we are producing a *set*. We do this using the Python type `set`. To apply shingling to each of our sentences we will define a `shingle` function:

In [3]:
def shingle(text: str, k: int):
    shingle_set = []
    for i in range(len(text) - k+1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)

In [4]:
a = shingle(a, k)
b = shingle(b, k)
c = shingle(c, k)
print(a)

{'pa', 'fi', 'is', 'e ', 'le', 'ti', 'st', 'sh', 'w ', 'ly', 'y ', 'ng', 'ew', 'in', 'th', 'yi', 'h ', ' s', ' f', 'sp', 'ce', 'ta', 'he', ' b', 'on', ' t', 'fl', 'g ', 'by', 'at', 'ac', 'io'}


Now that we have our three shingles we create a shingle vocabulary by create a `union` between all three sets.

In [5]:
vocab = list(a.union(b).union(c))
print(vocab)

['e ', 'nd', 'pe', 'ri', 'ou', 'it', 'hi', 'st', ' e', 'ly', 'ew', 'a ', 'ie', 'ks', 'et', 'yi', 'h ', 'sp', 'lo', ' b', 'ha', 'on', ' t', 'ma', 'gu', 'br', 'pa', 'an', 'te', 'tc', 'as', 'f ', 'le', 'ti', 'to', 'fe', 'ed', 'er', 'ol', 'u ', 'of', 'ca', 'na', ' f', 'ea', 'fl', 'n ', 'dy', 's ', 'ig', 'yn', 'wi', 'we', ' a', 'll', 'ur', 'yo', ' n', ' o', 'sh', 'w ', 't ', ' p', 'y ', 'ng', 'am', 'in', 'di', 'th', ' s', 'mi', 'ce', ' c', 'ta', 'ar', 'at', 'si', 'by', 'ow', 'io', 'fi', 'is', 'il', 'ck', 're', 'po', ' d', 'ad', 'r ', 'rm', 'o ', 'l ', 'al', 'ch', 'no', 'ic', 'he', ' w', 'g ', 'd ', 'ac', ' y', 'ot']


Using this vocab we can create one-hot encoded sparse vectors to represent our shingles.

In [6]:
a_1hot = [1 if x in a else 0 for x in vocab]
b_1hot = [1 if x in b else 0 for x in vocab]
c_1hot = [1 if x in c else 0 for x in vocab]
print(a_1hot)

[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0]


So we now have one-hot encoded sparse vectors we can move onto *minhashing*.

## Minhashing

Minhashing is the next step in our process. After creating our shingle sets we use minhashing to create *signatures* from those sets.

To create our minhashing function we build several hash functions, each will randomly count from `1` to `len(vocab) + 1` - creating a *random* vector:

In [7]:
hash_ex = list(range(1, len(vocab)+1))
print(hash_ex)  # we haven't shuffled yet

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103]


In [8]:
from random import shuffle

shuffle(hash_ex)
print(hash_ex)

[40, 56, 33, 70, 78, 65, 10, 34, 8, 21, 29, 53, 12, 20, 22, 28, 82, 95, 48, 73, 90, 94, 41, 1, 42, 15, 97, 35, 23, 37, 31, 18, 4, 11, 16, 49, 13, 3, 62, 54, 85, 71, 61, 52, 76, 84, 36, 9, 93, 17, 69, 45, 99, 50, 91, 102, 67, 79, 89, 44, 88, 30, 81, 58, 66, 2, 80, 19, 55, 39, 6, 46, 75, 92, 72, 25, 27, 96, 63, 38, 74, 100, 26, 60, 83, 24, 87, 98, 101, 64, 32, 14, 47, 86, 57, 7, 43, 59, 103, 68, 77, 51, 5]


We now have a randomized list of integers which we can use in creating our *hashed* signatures. What we do now is begin counting from `1` through to `len(vocab) + 1`, extracting the position of this number in our new `hash_ex` list, like so (for the first few integers):

In [9]:
for i in range(1, 5):
    print(f"{i} -> {hash_ex.index(i)}")

1 -> 23
2 -> 65
3 -> 37
4 -> 32


What we do with this is count up from `1` to `len(vocab) + 1` and find if the resultant `hash_ex.index(i)` position in our one-hot encoded vectors contains a positive value (`1`) in that position, like so:

In [10]:
for i in range(1, len(vocab)+1):
    idx = hash_ex.index(i)
    signature_val = a_1hot[idx]
    print(f"{i} -> {idx} -> {signature_val}")
    if signature_val == 1:
        print('match!')
        break

1 -> 23 -> 0
2 -> 65 -> 0
3 -> 37 -> 0
4 -> 32 -> 1
match!


That gives us a first signature value of `98`. But this is just a single value, and it takes many values to create a signature (100 for example).

So, how to we generate these other values? By using more hash functions! Let's generate a set of hash functions to create a signature vector of length `20`.

hash_funcs = []

for _ in range(20)

In [11]:
hash_funcs = []

for _ in range(20):
    hash_ex = list(range(1, len(vocab)+1))
    shuffle(hash_ex)
    hash_funcs.append(hash_ex)

for i in range(3):
    print(f"hash function {i+1}:")
    print(hash_funcs[i])

hash function 1:
[25, 46, 94, 48, 79, 36, 58, 34, 99, 61, 91, 8, 16, 93, 20, 97, 27, 53, 23, 4, 2, 59, 33, 83, 31, 81, 65, 71, 66, 100, 80, 74, 7, 70, 55, 10, 102, 26, 98, 85, 60, 39, 87, 90, 5, 73, 11, 88, 72, 21, 101, 54, 12, 1, 18, 49, 57, 13, 67, 78, 75, 51, 50, 64, 42, 19, 30, 29, 77, 9, 56, 38, 86, 41, 47, 3, 76, 96, 95, 37, 17, 22, 32, 43, 45, 15, 92, 63, 84, 103, 62, 68, 40, 28, 52, 82, 89, 24, 69, 44, 35, 6, 14]
hash function 2:
[101, 39, 40, 52, 56, 9, 92, 46, 51, 77, 3, 89, 59, 31, 87, 32, 76, 26, 71, 79, 93, 75, 64, 55, 73, 81, 1, 34, 83, 90, 80, 86, 74, 24, 84, 96, 58, 97, 2, 14, 13, 102, 67, 78, 54, 57, 95, 65, 49, 42, 25, 98, 47, 28, 27, 63, 17, 12, 11, 15, 60, 10, 53, 21, 99, 37, 8, 61, 4, 38, 35, 100, 70, 94, 5, 48, 33, 7, 62, 85, 103, 20, 30, 68, 16, 50, 88, 18, 43, 23, 44, 19, 29, 36, 72, 22, 45, 66, 69, 91, 6, 41, 82]
hash function 3:
[34, 43, 92, 68, 13, 50, 39, 56, 97, 31, 79, 102, 3, 85, 80, 36, 30, 47, 98, 15, 63, 101, 100, 37, 44, 27, 12, 46, 94, 99, 40, 4, 17,

We're only showing the first three hash functions here - we have *20* in total. To create our signatures we simply process each one-hot vector through each hash function, appending the output value to our signature for that vector.

In [12]:
signature = []

for func in hash_funcs:
    for i in range(1, len(vocab)+1):
        idx = func.index(i)
        signature_val = a_1hot[idx]
        if signature_val == 1:
            signature.append(idx)
            break

print(signature)

[75, 26, 64, 17, 81, 68, 17, 7, 64, 43, 69, 10, 10, 73, 73, 22, 15, 79, 0, 0]


And there we have our minhash produced signature for `a`. Let's clean up the code and formalize the process a little.

In [13]:
def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(1, size+1))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

# we create 20 minhash vectors
minhash_func = build_minhash_func(len(vocab), 20)

In [14]:
def create_hash(vector: list):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_func:
        for i in range(1, len(vocab)+1):
            idx = func.index(i)
            signature_val = vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature

In [15]:
# now create signatures
a_sig = create_hash(a_1hot)
b_sig = create_hash(b_1hot)
c_sig = create_hash(c_1hot)

print(a_sig)
print(b_sig)
print(c_sig)

[68, 80, 16, 77, 22, 17, 96, 43, 100, 68, 60, 9, 68, 22, 26, 10, 69, 45, 66, 80]
[54, 101, 54, 91, 22, 55, 47, 25, 56, 56, 70, 91, 55, 39, 94, 1, 57, 65, 66, 101]
[68, 84, 54, 85, 22, 55, 20, 43, 46, 49, 70, 85, 55, 46, 54, 44, 69, 65, 66, 90]


We now have our three minhashed signatures! These signatures, despite being seemingly randomized, will on average have the very similar Jaccard similarity values as our previous sparse vectors. We have reduced the dimensionality of our vectors significantly - but maintained the same information!


In [16]:
def jaccard(a: set, b: set):
    return len(a.intersection(b)) / len(a.union(b))

**a** should have lower similarity with **b** and **c**:

In [17]:
jaccard(a, b), jaccard(set(a_sig), set(b_sig))

(0.14814814814814814, 0.06896551724137931)

In [18]:
jaccard(a, c), jaccard(set(a_sig), set(c_sig))

(0.22093023255813954, 0.18518518518518517)

And **b** and **c** should share much greater simliarity:

In [19]:
jaccard(b, c), jaccard(set(b_sig), set(c_sig))

(0.45652173913043476, 0.24)

We're now ready to move onto the LSH process.

# Locality Sensitive Hashing

The approach we will be taking in this notebook is break our signature vector into multiple *bands*, creating several sub-vectors.

We then hash each of these sub-vectors into a set of buckets, if we find that two sub-vectors from two signature vectors collide (end up in the same hash bucket) we take the two *full* signature vectors as *candidate pairs* - which we then compare in full with a similarity metric (like Jaccard similarity, cosine similarity, etc).

There is no *'set'* way to hash our signature vectors, and in-fact the simplest approach is to check for equivalence across sub-vectors, which will be our approach.

First, we must define the number of buckets `b` we would like to create. It's important to note that each bucket must contain an equal number of rows `r` - and so our signature length must be divisible by `b`.

In [20]:
def split_vector(signature, b):
    assert len(signature) % b == 0
    r = int(len(signature) / b)
    # code splitting signature in b parts
    subvecs = []
    for i in range(0, len(signature), r):
        subvecs.append(signature[i : i+r])
    return subvecs

We'll start by splitting into 10 bands, creating rows of `2` - on the small side to be used in a genuine LSH function but good for our example (we'll explore different `r` and `b` values soon).

Let's start with our **b** and **c** vectors, which should *hopefully* match in *at least one* band.

In [21]:
band_b = split_vector(b_sig, 10)
band_b

[[54, 101],
 [54, 91],
 [22, 55],
 [47, 25],
 [56, 56],
 [70, 91],
 [55, 39],
 [94, 1],
 [57, 65],
 [66, 101]]

In [22]:
band_c = split_vector(c_sig, 10)
band_c

[[68, 84],
 [54, 85],
 [22, 55],
 [20, 43],
 [46, 49],
 [70, 85],
 [55, 46],
 [54, 44],
 [69, 65],
 [66, 90]]

Check if they match (we'll rewrite some of this into Numpy soon).

In [23]:
for b_rows, c_rows in zip(band_b, band_c):
    if b_rows == c_rows:
        print(f"Candidate pair: {b_rows} == {c_rows}")
        # we only need one band to match
        break

Candidate pair: [22, 55] == [22, 55]


And let's do the same for **a**.

In [24]:
band_a = split_vector(a_sig, 10)

In [25]:
for a_rows, b_rows in zip(band_a, band_b):
    if a_rows == b_rows:
        print(f"Candidate pair: {a_rows} == {b_rows}")
        # we only need one band to match
        break

In [26]:
for a_rows, c_rows in zip(band_a, band_c):
    if a_rows == c_rows:
        print(f"Candidate pair: {b_rows} == {c_rows}")
        # we only need one band to match
        print("matched!")
        break
    print("not matched")

not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched


Okay great, so even with this very simple implementation - we manage to identify sentences **b** and **c** and candidate pairs, and identify **a** as a non-candidate.

## Tuning LSH

We can visualize the probability of returning a candidate pair vs the similarity of the pair for different values of `r` and `b` (rows and bands respectively) like so:

In [27]:
def probability(s, r, b):
    # s: similarity
    # r: rows (per band)
    # b: number of bands
    return 1 - (1 - s**r)**b

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
results = pd.DataFrame({
    's': [],
    'P': [],
    'r,b': []
})

for s in np.arange(0.01, 1, 0.01):
    total = 100
    for b in [100, 50, 25, 20, 10, 5, 4, 2, 1]:
        r = int(total/b)
        P = probability(s, r, b)
        results = results.append({
            's': s,
            'P': P,
            'r,b': f"{r},{b}"
        }, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [29]:
sns.lineplot(data=results, x='s', y='P', hue='r,b')

NameError: name 'results' is not defined